In [ ]:
# vim: expandtab:ts=4:sw=4
from __future__ import absolute_import
import numpy as np
from . import kalman_filter
from . import linear_assignment
from . import iou_matching
from .track import Track


class Tracker:
    """
    This is the multi-target tracker.

    Parameters
    ----------
    metric : nn_matching.NearestNeighborDistanceMetric
        A distance metric for measurement-to-track association.
    max_age : int
        Maximum number of missed misses before a track is deleted.
    n_init : int
        Number of consecutive detections before the track is confirmed. The
        track state is set to `Deleted` if a miss occurs within the first
        `n_init` frames.

    Attributes
    ----------
    metric : nn_matching.NearestNeighborDistanceMetric
        The distance metric used for measurement to track association.
    max_age : int
        Maximum number of missed misses before a track is deleted.
    n_init : int
        Number of frames that a track remains in initialization phase.
    kf : kalman_filter.KalmanFilter
        A Kalman filter to filter target trajectories in image space.
    tracks : List[Track]
        The list of active tracks at the current time step.

    """

    def __init__(self, metric, max_iou_distance=0.7, max_age=30, n_init=3):
        self.metric = metric
        self.max_iou_distance = max_iou_distance
        self.max_age = max_age
        self.n_init = n_init

        self.kf = kalman_filter.KalmanFilter()
        self.tracks = []
        self._next_id = 1

    def predict(self):
        """Propagate track state distributions one time step forward.

        This function should be called once every time step, before `update`.
        """
        for track in self.tracks:
            track.predict(self.kf)

    def update(self, detections):
        """Perform measurement update and track management.

        Parameters
        ----------
        detections : List[deep_sort.detection.Detection]
            A list of detections at the current time step.

        """
        # Matching 해보자
        matches, unmatched_tracks, unmatched_detections = \
            self._match(detections)

        # match된 것만 kalman update
        for track_idx, detection_idx in matches:
            self.tracks[track_idx].update(
                self.kf, detections[detection_idx])
            
        # match되지 않은 track은 state delete로 바꿔줌
        for track_idx in unmatched_tracks:
            self.tracks[track_idx].mark_missed()
            
        # match되지 않은 detection box는 new track이기 때문에 kalman initiate로 바꾸고 track에 넣어줌
        for detection_idx in unmatched_detections:
            self._initiate_track(detections[detection_idx])
            
        # state가 delete가 아닌 track만 tracks에 저장
        self.tracks = [t for t in self.tracks if not t.is_deleted()]

        # 추적되는 track만 active_targets에 id저장
        active_targets = [t.track_id for t in self.tracks if t.is_confirmed()]
        
        # 확인되는 track만 self.sample에 {target:feature}로 저장
        features, targets = [], []
        for track in self.tracks:
            if not track.is_confirmed():
                continue
            features += track.features
            targets += [track.track_id for _ in track.features]
            track.features = [] #초기화
            
        # 제일 최근에 생긴 것부터 최대 객체 갯수(budget)만큼만 남기고 다 버림(budget=None이면 버리는 거 없음)
        self.metric.partial_fit( 
            np.asarray(features), np.asarray(targets), active_targets)

    def _match(self, detections):
        
        def gated_metrics(tracks, dets, track_indices, detection_indices):
            features = np.array([dets[i].feature for i in detection_indices])
            targets = np.array([tracks[i].track_id for i in track_indices])
            
            # 
            cost_matrix = self.metric.distance(features, targets)
            cost_matrix = linear_assignment.gate_cost_matrix(
                self.kf, cost_matrix, tracks, dets, track_indices,
                detection_indices)

            return cost_matrix

        # 확인되는 track과 되지 않는 track 나누기
        confirmed_tracks = [
            i for i, t in enumerate(self.tracks) if t.is_confirmed()]
        unconfirmed_tracks = [
            i for i, t in enumerate(self.tracks) if not t.is_confirmed()]

        # Associate confirmed tracks using appearance features. Re-ID과정
        # confirm된 것 중 matching_threshold보다 큰 distance : unmatched, 작을 때 : match
        matches_a, unmatched_tracks_a, unmatched_detections = \
            linear_assignment.matching_cascade(
                gated_metrics, self.metric.matching_threshold, self.max_age,
                self.tracks, detections, confirmed_tracks)
        
        # IoU과정
        # track 중 matching 뒤 업데이트 뒤 kalman필터로 처음 예측한 것들
        iou_track_candidates = unconfirmed_tracks + [
            k for k in unmatched_tracks_a if
            self.tracks[k].time_since_update == 1]
        
        # new_track아니고 match되지 않은 track만 unmatched_tracks_a에 저장
        unmatched_tracks_a = [
            k for k in unmatched_tracks_a if
            self.tracks[k].time_since_update != 1]
        
        # 확인되지 않았고 매칭되지는 않았지만 update횟수가 1번인 것들은 1e+5, 아닌 건 1-iou계산
        # 1-iou값이 작은 것들 matching
        matches_b, unmatched_tracks_b, unmatched_detections = \
            linear_assignment.min_cost_matching(
                iou_matching.iou_cost, self.max_iou_distance, self.tracks,
                detections, iou_track_candidates, unmatched_detections)
        
        # Re-Id, Iou로 match한 것들 합치기
        matches = matches_a + matches_b
        unmatched_tracks = list(set(unmatched_tracks_a + unmatched_tracks_b))
        return matches, unmatched_tracks, unmatched_detections  # match : index 튀어나옴 (track_idx, detection_idx)

    def _initiate_track(self, detection):
        mean, covariance = self.kf.initiate(detection.to_xyah())
        class_name = detection.get_class()
        self.tracks.append(Track(
            mean, covariance, self._next_id, self.n_init, self.max_age,
            detection.feature, class_name))
        self._next_id += 1
